In [1]:
from mip import *
from dataclasses import dataclass
import itertools
from collections import defaultdict
import math
from copy import copy

In [2]:
capacity_per_flight = 2

In [3]:
class LinearExpression:
    def __init__(self):
        self.variables = defaultdict(lambda: 0)
        self.constant = 0
        
    def __repr__(self):
        return f"LE with n_var: {len(self.variables)} and const: {self.constant}"
    
    def add_variable(self, variable, coefficient = 1):
        self.variables[variable] += coefficient
        if self.variables[variable] == 0:
            del self.variables[variable]

    def add_constant(self, constant):
        self.constant += constant
    
    def __copy__(self):
        le = LinearExpression()
        for variable, coefficient in self.variables.items():
            le.add_variable(variable, coefficient)
        le.add_constant(self.constant)
        return le
    
    def __sub__(self, other):
        le = copy(self)
        for variable, coefficient in other.variables.items():
            le.add_variable(variable, -1*coefficient)
        le.add_constant(-1*other.constant)
        return le

In [4]:
class EqualityConstraint:
    def __init__(self, lhs, rhs):
        assert isinstance(lhs, LinearExpression)
        assert isinstance(rhs, (float, int))
        self.rhs = rhs
        self.lhs = lhs

class LE_InequalityConstraint:
    """
    Less than or equal to inequality constraint
    """
    def __init__(self, lhs, rhs):
        assert isinstance(lhs, LinearExpression)
        assert isinstance(rhs, (float, int))
        self.rhs = rhs
        self.lhs = lhs
    

In [5]:
@dataclass(frozen=True)
class Hub:
    x_coordinate: int
    y_coordinate: int
    inventory: dict
        
    def __repr__(self):
        return f"hub ({self.x_coordinate}, {self.y_coordinate})"
    
    def __hash__(self):
        return id(self)
        
@dataclass(frozen=True)
class Customer:
    x_coordinate: int
    y_coordinate: int
    demand: dict
        
    def __repr__(self):
        return f"customer ({self.x_coordinate}, {self.y_coordinate})"
    
    def __hash__(self):
        return id(self)

@dataclass(frozen=True)
class Product:
    unique_id: int

In [6]:
def get_distance(place1, place2):
    return math.sqrt(abs(place1.x_coordinate - place2.x_coordinate)**2 + abs(place1.y_coordinate - place2.y_coordinate)**2)

In [7]:
def get_products():
    return [Product(0), Product(1)]

def get_hubs(products):
    return [Hub(-1,0, {products[0]: 0, products[1]: 2}), Hub(1,0,{products[0]: 2, products[1]: 0}), Hub(0,5,{products[0]: 0, products[1]: 0})]

def get_customers(products):
    return [Customer(0,10, {products[0]: 2, products[1]: 2})]

In [8]:
products = get_products()
hubs = get_hubs(products)
customers = get_customers(products)

In [9]:
@dataclass(frozen=True)
class IntegerVariable:
    name: str
    lower_bound: int = None
    upper_bound: int = None

    def __hash__(self):
        return id(self)


In [10]:
def get_n_flights_variables(customers, hubs):
    return {(customer, hub): IntegerVariable(name={f"number of flights from {hub} to {customer}"},
                                            lower_bound=0) 
            for customer, hub in itertools.product(customers, hubs)}

def get_n_products_move_variables(customers, hubs, products):
    return {(customer, hub, product): IntegerVariable(name={f"number of products of product type {product} from {hub} to {customer}"},
                                                     lower_bound=0,
                                                     upper_bound=customer.demand[product]) 
        for customer, hub, product in itertools.product(customers, hubs, products)}

def get_n_flights_hub_to_hub(hubs):
    return {(hub1, hub2): IntegerVariable(name={f"number of flights from {hub1} to {hub2}"},
                                            lower_bound=0) 
            for hub1, hub2 in itertools.product(hubs, hubs) if hub1 != hub2}

def get_n_products_move_hub_to_hub(hubs, products):
    return {(hub1, hub2, product): IntegerVariable(name={f"number of products of product type {product} from {hub1} to {hub2}"},
                                                     lower_bound=0)
        for hub1, hub2, product in itertools.product(hubs, hubs, products) if hub1 != hub2}

In [11]:
n_flights_variables = get_n_flights_variables(customers, hubs)
n_product_move_variables = get_n_products_move_variables(customers, hubs, products)
n_flights_hub_to_hub_variables = get_n_flights_hub_to_hub(hubs)
n_products_move_hub_to_hub_variables = get_n_products_move_hub_to_hub(hubs, products)

decision_variables = list(n_flights_variables.values())+ list(n_product_move_variables.values()) + list(n_flights_hub_to_hub_variables.values()) + list(n_products_move_hub_to_hub_variables.values())

In [12]:
def get_objective(n_flights_variables, n_flights_hub_to_hub_variables):
    le = LinearExpression()
    for customer, hub in itertools.product(customers, hubs):
        le.add_variable(variable=n_flights_variables[customer, hub], coefficient=get_distance(hub, customer))
    
    for hub1, hub2 in itertools.product(hubs, hubs):
        if hub1 != hub2:
            le.add_variable(variable=n_flights_hub_to_hub_variables[hub1, hub2], coefficient=get_distance(hub1, hub2))  
    return le

In [13]:
objective = get_objective(n_flights_variables, n_flights_hub_to_hub_variables)

In [14]:
def get_demand_constraints(n_product_move_variables):
    constraints = []
    for customer, product in itertools.product(customers, products):
        lhs = LinearExpression()
        for hub in hubs:
            lhs.add_variable(n_product_move_variables[(customer, hub, product)])
        constraints.append(EqualityConstraint(lhs=lhs, rhs=customer.demand[product]))
    return constraints

def get_supply_constraints(n_product_move_variables, n_products_move_hub_to_hub_variables):
    constraints = []
    for hub, product in itertools.product(hubs, products):
        le_1 = LinearExpression()
        for customer in customers:
            le_1.add_variable(n_product_move_variables[(customer, hub, product)])
            
        le_2 = LinearExpression()
        for hub_ in hubs:
            if hub_ != hub:
                le_2.add_variable(n_products_move_hub_to_hub_variables[(hub_, hub, product)], 1)
                le_2.add_variable(n_products_move_hub_to_hub_variables[(hub, hub_, product)], -1)

        constraints.append(LE_InequalityConstraint(lhs=le_1-le_2, rhs=hub.inventory[product]))
    return constraints

def get_trips_constraints(n_product_move_variables, n_flights_variables):
    constraints = []
    for customer, hub in itertools.product(customers, hubs):
        le_1 = LinearExpression()
        for product in products:
            le_1.add_variable(n_product_move_variables[(customer, hub, product)])
        
        le_2 = LinearExpression()
        le_2.add_variable(n_flights_variables[(customer, hub)], capacity_per_flight)
        
        constraints.append(LE_InequalityConstraint(lhs=le_1 - le_2, rhs=0))
    return constraints

def get_trips_hub_to_hub_constraints(n_products_move_hub_to_hub_variables, n_flights_hub_to_hub_variables):
    constraints = []
    for hub1, hub2 in itertools.product(hubs, hubs):
        if hub1 != hub2:
            le_1 = LinearExpression()
            for product in products:
                le_1.add_variable(n_products_move_hub_to_hub_variables[(hub1, hub2, product)])

            le_2 = LinearExpression()
            le_2.add_variable(n_flights_hub_to_hub_variables[(hub1, hub2)], capacity_per_flight)

            constraints.append(LE_InequalityConstraint(lhs=le_1 - le_2, rhs=0))
    return constraints

In [15]:
demand_constraints = get_demand_constraints(n_product_move_variables)
supply_constraints = get_supply_constraints(n_product_move_variables, n_products_move_hub_to_hub_variables)
trips_constraints = get_trips_constraints(n_product_move_variables, n_flights_variables)
trips_hub_hub_constraints = get_trips_hub_to_hub_constraints(n_products_move_hub_to_hub_variables, n_flights_hub_to_hub_variables)
constraints = demand_constraints + supply_constraints + trips_constraints + trips_hub_hub_constraints

# Make model

In [16]:
class MipModel:

    def __init__(
        self,
        decision_variables,
        constraints,
        objective,
    ):
        self.m = Model(sense=MINIMIZE, solver_name=CBC)
        self.model_variables = self._add_variables_to_model(decision_variables)
        
        self._add_constraints_to_model(constraints)
        self._add_objective_to_model(objective)
    
    def _add_objective_to_model(self, objective):           
        self.m.objective = minimize(
            xsum(coeff * self.model_variables[var] for var, coeff in objective.variables.items())
        )

    
    def _add_constraints_to_model(self, constraints):
        for constraint in constraints:
            if isinstance(constraint, LE_InequalityConstraint):
                self.m += (
                    xsum(
                        coeff * self.model_variables[var]
                        for var, coeff in constraint.lhs.variables.items()
                    )
                    <= constraint.rhs
                )

            if isinstance(constraint, EqualityConstraint):
                self.m += (
                    xsum(
                        coeff * self.model_variables[var]
                        for var, coeff in constraint.lhs.variables.items()
                    )
                    == constraint.rhs
                )

    def _add_variables_to_model(self, decision_variables):
        model_variables = {}
        for decision_variable in decision_variables:
            if isinstance(decision_variable, IntegerVariable):
                model_variables[decision_variable] = self.m.add_var(var_type=INTEGER, lb=decision_variable.lower_bound)
            else:
                raise Exception(f"Do not recognize this variable {decision_variable}")
        return model_variables
    
    def solve(self):
        self.status = self.m.optimize()
        return {
            name: int(np.round(model_variable.x))
            for name, model_variable in self.model_variables.items()
        }


In [17]:
mip_model = MipModel(decision_variables, constraints, objective)

In [18]:
solution = mip_model.solve()

In [19]:
for variable, value in solution.items():
    if value != 0:
        print(variable, value)

IntegerVariable(name={'number of flights from hub (1, 0) to customer (0, 10)'}, lower_bound=0, upper_bound=None) 1
IntegerVariable(name={'number of products of product type Product(unique_id=0) from hub (1, 0) to customer (0, 10)'}, lower_bound=0, upper_bound=2) 2
IntegerVariable(name={'number of products of product type Product(unique_id=1) from hub (1, 0) to customer (0, 10)'}, lower_bound=0, upper_bound=2) 2
IntegerVariable(name={'number of flights from hub (-1, 0) to hub (1, 0)'}, lower_bound=0, upper_bound=None) 1
IntegerVariable(name={'number of products of product type Product(unique_id=1) from hub (-1, 0) to hub (1, 0)'}, lower_bound=0, upper_bound=None) 2
